In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import ttest_ind
from dipy.core.sphere import disperse_charges, HemiSphere
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.sims.voxel import multi_tensor, multi_tensor_odf, single_tensor_odf
from dipy.viz import window, actor
from dipy.reconst.csdeconv import mask_for_response_ssst, response_from_mask_ssst
import nibabel as nib

#SCRIPT

import nibabel as nib
import numpy as np
from numpy.linalg import norm
from scipy import stats
from math import *
import sys, glob, xlwt, os.path, re

In [7]:
# Open the file in read mode
with open('real_data/hardi-scheme.bvec.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an empty matrix to store the values
real_bvecs = []

# Iterate through each line
for line in lines:
    # Split the line into individual values
    values = line.split() 
    # Convert the values to integers and append to the matrix
    real_bvecs.append([float(value) for value in values])

# Convert the list of lists into a 2D NumPy array
real_bvecs = np.array(real_bvecs).T


# Open the file in read mode
with open('real_data/hardi-scheme.bval.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an empty matrix to store the values
real_bvals = []

# Iterate through each line
for line in lines:
    # Split the line into individual values
    values = line.split() 
    # Convert the values to integers and append to the matrix
    real_bvals.append([float(value) for value in values])

# Convert the list of lists into a 2D NumPy array
real_bvals = np.array(real_bvals).reshape(-1)

gtab = gradient_table(real_bvals, real_bvecs)

mask = nib.load('real_data/mask.nii').get_fdata().astype(bool)
white_mask = nib.load('real_data/white_matter_mask.nii').get_fdata().astype(bool)

hardi_snr10 = nib.load('real_data/DWIS_hardi-scheme_SNR-10.nii').get_fdata()
hardi_snr20 = nib.load('real_data/DWIS_hardi-scheme_SNR-20.nii').get_fdata()
hardi_snr30 = nib.load('real_data/DWIS_hardi-scheme_SNR-30.nii').get_fdata()

hardi_snr10_masked = hardi_snr10[mask]
hardi_snr20_masked = hardi_snr20[mask]
hardi_snr30_masked = hardi_snr30[mask]
hardi_snr10_masked *= (100/hardi_snr10_masked[0][0])
hardi_snr20_masked *= hardi_snr20_masked * (100/hardi_snr20_masked[0][0])
hardi_snr30_masked *= hardi_snr30_masked * (100/hardi_snr30_masked[0][0])

hardi_snr10_white_masked = hardi_snr10[white_mask]
hardi_snr20_white_masked = hardi_snr20[white_mask]
hardi_snr30_white_masked = hardi_snr30[white_mask]
hardi_snr10_white_masked *= (100/hardi_snr10_white_masked[0][0])
hardi_snr20_white_masked *= (100/hardi_snr20_white_masked[0][0])
hardi_snr30_white_masked *= (100/hardi_snr30_white_masked[0][0])

hardi_white_masked_all = np.vstack((hardi_snr10_white_masked, hardi_snr20_white_masked, hardi_snr30_white_masked))
np.random.seed(42)
np.random.shuffle(hardi_white_masked_all)

S = np.load("synthetic_data/S.npy")
S_noiseless = np.load("synthetic_data/S_noiseless.npy")
F = np.load('synthetic_data/F.npy')
angle_pairs = pd.read_pickle("synthetic_data/angle_pairs.pkl")

In [19]:
sphere = get_sphere('symmetric724')
response_mask = mask_for_response_ssst(gtab, hardi_snr30, roi_radii=20, fa_thr=0.7)
response, ratio = response_from_mask_ssst(gtab, hardi_snr30, response_mask)
print(response)
print(ratio)
scene = window.Scene()
evals = response[0]
evecs = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]).T

response_odf = single_tensor_odf(sphere.vertices, evals, evecs)
# transform our data from 1D to 4D
response_odf = response_odf[None, None, None, :]
response_actor = actor.odf_slicer(response_odf, sphere=sphere,
                                  colormap='plasma')
scene.add(response_actor)
window.show(scene)
scene.rm(response_actor)

(array([0.00164641, 0.00038227, 0.00038227]), 32767.0)
0.23218691030646116


UNDERSTANDING THE "compute_local_metrics.py" SCRIPT

In [24]:
### load ground-truth directions
################################
print("-> opening ground-truth...")

niiGT = nib.load('real_data/ground-truth-peaks.nii.gz')
niiGT_hdr = niiGT.header
niiGT_img = niiGT.get_fdata()

niiGT_dim = niiGT_hdr.get_data_shape()

nx = niiGT_dim[0]
ny = niiGT_dim[1]
nz = niiGT_dim[2]

print("[OK]\n")

-> opening ground-truth...
[OK]


In [29]:
### load mask of interest
#########################
print("-> opening binary mask of interest..."),

niiMASK = nib.load('real_data/white_matter_mask.nii.gz')
niiMASK_hdr = niiMASK.header
niiMASK_img = niiMASK.get_fdata() # 0-1 mask

niiMASK_dim = niiMASK_hdr.get_data_shape()

niiMASK_idx = niiMASK_img==1 #Boolean mask

print("[OK]\n")

-> opening binary mask of interest...
